# **Near Field Scan Script for Aaron's Quickturn Antenna**

Tunes antenna using two Arduinos + two DAC boards

### Import modules

In [4]:
import sys
sys.path.append('C:/Users/smithlab/Documents/Acquisition-Code/Modules')
import numpy as np
from matplotlib import pyplot as plt
import VNA_control as vc
import stage_control as sc
import probe_deconvolve as pd
import propagation_functions as pf
import scipy.constants
import scipy.io
import time
import datetime
import mainPyDAC2_module_multiple_Aaron as tune
import numpy.matlib
import csv

def load_list(filename):
    # loads in CSV file, supporting lists (i.e. CSVs exported from up to two-dimensional arrays)
    list1 = []
    with open(filename, 'r', newline='\n') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            list1.append(row)

    list1 = [[float(v) for v in r] for r in list1]
    return list1

C     = scipy.constants.c
EPS_0 = scipy.constants.epsilon_0 #C^2/(N*m^2)
MU_0  = scipy.constants.mu_0    #m kg s^-2 A^-2
mm = 0.001
cm    = 0.01
GHz   = 1.0E9

### Tuning state generation

In [8]:
ts = []
for i in range(240):
    t = np.zeros(240)
    t[i] = 2.5          # creating tuning states with one element at a time set to 2.5V
    ts.append(t)


### General scan parameters

In [6]:
a_probe = 22.86
b_probe = 10.16

xrange = 1000 # x length of stage, mm
yrange = 1000 # y length of stage, mm

fstart = 9E9
fstop = 10E9
NumFreqs = 51
NumApMasks = len(ts)
f = np.linspace(fstart, fstop, NumFreqs)
lam_min = C/fstop
dstep = lam_min/2 * 1000 # mm

IFBW = 1000
calfile = 'Aaron_21-11-05'
power = 0 # dBm
sParMeas = 'S12'
speedmms = 25       # don't change this

print('IFBW = {} \nCalibration File = {} \nPower = {} dBm \nMeasuring {}'.format(IFBW, calfile, power, sParMeas))

IFBW = 1000 
Calibration File = Aaron_21-11-05 
Power = 0 dBm 
Measuring S12


### Initialize instruments

In [11]:
## Stage
g = sc.stage_start('169.254.225.173')
sc.stage_home(g,speedmms)

## Arduino
numDACs = 30
serial1 = tune.initializeArduino(numDACs, "COM9")
serial2 = tune.initializeArduino(numDACs, "COM10")

## VNA
vna = vc.instrument_open('TCPIP0::169.254.187.153::5025::SOCKET')
vc.VNA_initiate(vna, NumFreqs, fstart*1E-9, fstop*1E-9, IFBW, power, 'S12', calfile=calfile)

Agilent Technologies,N5222A,MY51421573,A.09.90.10
Initiating VNA...

Done



### Configure save path

In [12]:
savename = 'Quickturn_Antenna_Element_Index_Test'
savepath = 'C:/Users/smithlab/Documents/Near_Field_Scans/'

### Scan

#### *Set scan positions*

In [24]:
r_list = 1000 * np.asarray(load_list(r'C:\Users\User\Documents\KiCad\Projects\Aaron_Prototype_Final\Design Documents\Design_Data\r_list.csv'))
r_list = r_list - r_list[0,None,:]      # origin is element 1 position

plt.figure()
plt.scatter(r_list[:,0], r_list[:,1])
plt.show()

measurements = np.zeros((r_list.shape[0], NumFreqs, NumApMasks), dtype=np.complex128) # measurements with E field of probe oriented in x direction
stops = r_list.shape[0]

#### *Move to new origin*

In [35]:
center_x = -xrange/2
center_y = -yrange/2
pos_x = -70  # adjust to change x zero position
pos_y = -40  # adjust to change y zero position

sc.move_to_absolute(g,speedmms,0,0,center_x+pos_x,center_y+pos_y)

(-570.0, -540.0)

#### *Set origin*

In [37]:
defZeroInXsteps, defZeroInYsteps = sc.zero_axes(g)

#### *Start scan - x-polarization*

In [41]:
sc.move_to_absolute(g,speedmms,defZeroInXsteps,defZeroInYsteps,r_list[0,0],r_list[0,1]) # move to start position

(-202.5388, -147.5388)

In [23]:
t = time.time()
stopscomp = 0

for ii in range(r_list.shape[0]):

    sc.move_to_absolute(g,speedmms,defZeroInXsteps,defZeroInYsteps,r_list[ii,0],r_list[ii,1])

    for jj in range(NumApMasks):
        tune.testProgram30DAC(serial1, ts[jj])
        tune.testProgram30DAC(serial2, ts[jj])
        time.sleep(0.01)
        measurements[ii,:,jj] = vc.VNA_read(vna,sParMeas)

    stopscomp += 1
    timere = (stops - stopscomp) * (time.time()-t)/3600
    if stopscomp%5==0:
        print('Est. time remaining: {} hours'.format(timere))
    t = time.time()

### Save data

In [48]:
scipy.io.savemat('{}{}_{}.mat'.format(savepath, savename, datetime.datetime.now().strftime('%Y-%m-%d')),
                                      {'r_list': r_list,
                                       'f': f,
                                       'ts': ts,
                                       'measurements': measurements})

### Find element index mapping

In [29]:
f0 = 9.75E9
peaks = np.abs(measurements[:,np.argmin(np.abs(f0 - f)),:])
element_index = np.argmax(peaks, axis=1)
### NEED TO FIX DEAD ELEMENT INDICES

# Example: have desired set of 160 voltages alpha_v, and tuning vector t of length 240.
# Then set t[element_index] = alpha_v

t_test = np.zeros(10)
element_index_test = np.array([2, 1, 7, 9, 5]) # 5 elements
alpha_v_test = np.array([1, 2, 3, 4, 5])

t_test[element_index_test] = alpha_v_test

print(t_test)

(160,)